In [32]:
import pandas as pd
import pickle
import numpy as np
from SOM_plus_clustering.utils import euc_distance
import seaborn as sns

In [38]:
som_model = pickle.load(open('models/som_model.models', 'rb'))
som_model
pca = pickle.load(open('models/pca.models', 'rb'))

In [18]:
cluster_center = som_model.cluster_center_

In [24]:
dist_matrix = list()
for i in cluster_center:
    rows = list()
    for j in cluster_center:
        rows.append(euc_distance(i,j))
    dist_matrix.append(rows)
dist_matrix = np.array(dist_matrix)

In [29]:
class minmaxscaler:
    def __init__(self, multiplier_value) -> None:
        self.multiplier = multiplier_value
        pass
    def fit(self, X):
        self.min_value = np.min(X)
        self.max_value = np.max(X)
        self.scale = 1/(self.max_value - self.min_value)
    def fit_transform(self, X):
        self.min_value = np.min(X)
        self.max_value = np.max(X)
        self.scale = 1/(self.max_value - self.min_value)
        return [(i - self.min_value)*self.scale*self.multiplier for i in X]
    def inverse_transform(self, X):
        return [i/self.scale/self.multiplier+self.min_value for i in X]

In [31]:
scaler = minmaxscaler(1)
scaled_dist = scaler.fit_transform(X=dist_matrix)

In [36]:
pd.DataFrame(dist_matrix).to_csv("Stats/som_distance_matrix.csv")
pd.DataFrame(dist_matrix).to_excel("Stats/som_distance_matrix.xlsx")

In [60]:
data = pd.read_csv("labeled_data.csv")
columns = data.columns[:-1]

In [58]:
cluster_data = pd.DataFrame(pca.inverse_transform( som_model.cluster_center_), columns=columns)
cluster_data["weight"] = scaler.inverse_transform(list(cluster_data["weight"]))
converted_cluster = cluster_data[cluster_data > 0.01].dropna(axis=1)
converted_cluster

,"('N', 'C', 1)","('S', 'H', 1)","('C', 'S', 1.5)","('O', 'C', 1.5)","('C', 'O', 2)","('C', 'O', 1)","('C', 'C', 1)","('C', 'C', 1.5)","('C', 'H', 1)",C,...,O,amines,ether,aldehyde,sulfide,aromatic,cycle,weight,logP,SAS
0,3.506771,0.189581,0.368430,0.156544,0.874080,2.315926,0.677827,9.575355,12.021189,13.436372,...,2.776937,1.407153,1.066805,0.874080,0.082816,1.932883,0.013026,187.430863,2.122674,2.318506
1,7.642421,0.049507,0.094971,0.148457,1.317401,0.455673,18.147597,1.215907,34.955176,21.979642,...,1.470982,2.509580,0.186793,1.317401,0.029236,0.366993,3.418159,280.898474,2.338108,4.401543
2,0.644403,0.036420,0.073611,0.189771,0.599767,7.832370,7.492921,6.119832,26.169210,16.944302,...,5.717052,0.337634,3.646422,0.599767,0.034070,1.020416,1.017700,225.663360,1.889261,3.242395
3,13.453086,0.021078,0.018463,0.028444,2.194707,0.635177,8.119276,1.112993,22.877841,15.945253,...,2.347164,4.334019,0.373834,2.194707,0.023709,0.088109,2.724793,266.444221,0.471847,3.101234


In [59]:
converted_cluster.to_csv("reduced_cluster_center_data.csv")
converted_cluster.round(3).to_excel("reduced_cluster_center_data.xlsx")

In [75]:
data[converted_cluster.columns].corr().round(3).to_csv("Stats/correlation.csv")
data[converted_cluster.columns].corr().round(3).to_excel("Stats/correlation.xlsx")
data[converted_cluster.columns].corr().abs().mean().to_csv("Stats/average_absolute_correlation.csv")
data[converted_cluster.columns].corr().abs().mean().to_excel("Stats/average_absolute_correlation.xlsx")

In [81]:
pd.DataFrame(pca.explained_variance_ratio_, columns=["Variance Ratio"]).round(3).to_csv("Stats/variance_ratio.csv")
pd.DataFrame(pca.explained_variance_ratio_, columns=["Variance Ratio"]).round(3).to_excel("Stats/variance_ratio.xlsx")